In [54]:
import torch
import torch.nn as nn
from torch.nn import functional as F

In [55]:
device ='cuda' if torch.cuda.is_available() else 'cpu'
print(device)

cuda


In [56]:
block_size= 8
batch_size= 4
max_iters = 10000
learning_rate = 3e-4
eval_iters =1000


In [57]:
with open("wizard_of_oz.txt",'r',encoding='utf-8') as f:
    text= f.read()

chars = sorted(set(text))
print(chars)
vocab_size=len(chars)

['\n', ' ', '!', '"', '&', "'", '(', ')', '*', ',', '-', '.', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', ':', ';', '?', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', '[', ']', '_', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', '\ufeff']


In [58]:
string_to_int = {ch : i for i,ch in enumerate(chars)}
int_to_string = { i:ch for i,ch in enumerate(chars)}
encode= lambda s: [string_to_int[c] for c in s]
decode = lambda l: ''.join([int_to_string[i] for i in l])

data = torch.tensor(encode(text), dtype=torch.long)

#encoded_hello=encode('hello')
#decoded_hello= decode(encoded_hello)
#print(encoded_hello)
#print(decoded_hello)

print(data[:100])


tensor([80,  1,  1, 28, 39, 42, 39, 44, 32, 49,  1, 25, 38, 28,  1, 44, 32, 29,
         1, 47, 33, 50, 25, 42, 28,  1, 33, 38,  1, 39, 50,  0,  0,  1,  1, 26,
        49,  0,  0,  1,  1, 36, 11,  1, 30, 42, 25, 38, 35,  1, 26, 25, 45, 37,
         0,  0,  1,  1, 25, 45, 44, 32, 39, 42,  1, 39, 30,  1, 44, 32, 29,  1,
        47, 33, 50, 25, 42, 28,  1, 39, 30,  1, 39, 50,  9,  1, 44, 32, 29,  1,
        36, 25, 38, 28,  1, 39, 30,  1, 39, 50])


In [59]:
n= int(0.8*len(data))
train_data = data[:n]
val_data =data[n:]

def get_batch(split):
    data= train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,))
    x= torch.stack([data[i:i+block_size] for i in ix])
    y= torch.stack([data[i+1:i+block_size+1] for i in ix])
    x,y = x.to(device), y.to(device)
    return x,y

x,y = get_batch('train')
print("inputs:\n",x,"\ntargets:\n",y)
    

inputs:
 tensor([[ 5, 72,  1, 54,  1, 78, 58, 65],
        [71, 54, 73, 62, 68, 67, 22,  1],
        [67,  1, 73, 61, 58,  1, 76, 62],
        [ 0,  3, 49, 68, 74,  1, 66, 74]], device='cuda:0') 
targets:
 tensor([[72,  1, 54,  1, 78, 58, 65, 65],
        [54, 73, 62, 68, 67, 22,  1, 44],
        [ 1, 73, 61, 58,  1, 76, 62, 67],
        [ 3, 49, 68, 74,  1, 66, 74, 72]], device='cuda:0')


In [60]:

x= train_data[:block_size]
y= train_data[1:block_size+1]

for t in range(block_size):
    context= x[:t+1]
    target= y[t]
    print("when input is ", context, 'target is ',target)

when input is  tensor([80]) target is  tensor(1)
when input is  tensor([80,  1]) target is  tensor(1)
when input is  tensor([80,  1,  1]) target is  tensor(28)
when input is  tensor([80,  1,  1, 28]) target is  tensor(39)
when input is  tensor([80,  1,  1, 28, 39]) target is  tensor(42)
when input is  tensor([80,  1,  1, 28, 39, 42]) target is  tensor(39)
when input is  tensor([80,  1,  1, 28, 39, 42, 39]) target is  tensor(44)
when input is  tensor([80,  1,  1, 28, 39, 42, 39, 44]) target is  tensor(32)


In [61]:
@torch.no_grad()
def estimate_loss():
    out= {}
    model.eval()
    for split in ['train','val']:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            X,Y = get_batch(split)
            logits,loss= model(X,Y)
            losses[k] = loss.item()
        out[split] = losses.mean()
    model.train()
    return out

In [62]:
class BigramLanguageModel(nn.Module):
    def __init__(self,vocab_size):
        super().__init__()
        self.token_embedding_table = nn.Embedding(vocab_size,vocab_size)    #embedding table of the bigrams 
        #Eg: if 'a followed by c' is more likely to accur than 'a followed by d', then the value of box along row representing 'a' and 
        #column representing 'c' will be greater than the value of box along row representing 'a' and column representating 'd' in
        #the embedding table (thats why we need vocab_size x vocab_size table to include all such pairs (bigrams))
        
    def forward(self, index,targets=None):
        logits = self.token_embedding_table(index)   #the normalized probability distribution of the bigrams 
        

        if targets is None:
            loss= None
        else:
            #Batch dimension, Time dimension (we dont know yet), Channels or Class dimension - vocab size
            B, T, C = logits.shape
            
            logits = logits.view(B*T, C)  #blend B and T (alter shape of the tensor according to the input expectation of cross_entropy() using view() ) 
            targets = targets.view(B*T)       
            loss = F.cross_entropy(logits, targets)
        return logits, loss

    def generate(self, index, max_new_tokens):
        #index is the (B,T) array of indices in the current context

        for i in range(max_new_tokens):
            #get the prediction
            logits, loss = self.forward(index)

            #focus on last time step alone
            logits = logits[:,-1,:] #becomes (B,C)

            #apply softmax to get probabilities
            probs = F.softmax(logits,dim=-1) 
            #sample from the distribution
            index_next = torch.multinomial(probs, num_samples=1) #(B,1)
            #append sampled index to the running sequence
            index=torch.cat((index,index_next),dim=1) #(B,T+1)
        return index
model = BigramLanguageModel(vocab_size)
m = model.to(device)

context = torch.zeros((1,1), dtype = torch.long, device=device)
generated_chars = decode(m.generate(context,max_new_tokens=500)[0].tolist())
print(generated_chars)


        


k7XkBKDxn5geal'Vn-3MHi7-a0uuDhqOboh?tg9,k8Q!Bhj[uYqUbMP&m)SWly'JkXPi7'.XWCy]tKEUMHCdj(Qtcp1zg0Ibdizsw2b﻿sET)wS!V,fFJ6RZsc]"ZNrJs(90Oux4Q&2n)THE(6Z*fZF.gWG;7eu7pGgij*ZeQ6a;A.c]60']3Rkm)8"2?XblYuiEo,Dwa _,XZS]O[W﻿9WCmrxx[1GplJwaJmnN;j0'iFVr"-Sgu5nJk[(WKgvR3mZp!VLv?'b8bdH*J.O O'waYpeKP&p-z?54;nEHEzNH?3Qsu4-i7pAKmFwDghC2no[:GP)o
_0z"IGhhIXb﻿J6cLvqjUD:N7l&uQU;scb8"&!SokPK39:O"IrXMdzkNF0!_o
rGC!STjFVUf_ShXb h"lhjm6ScLv:g1hITekOfNnTtvl9g17eQShjS&EIB SHzP.)o?a1nH&﻿iC
4Vg-gWkXc]P6QYySH5A-.::3 _V7sbs﻿(b"!


In [68]:
optimizer = torch.optim.AdamW(model.parameters(), lr= learning_rate)

for iter in range(max_iters):
    if iter%eval_iters == 0:
        losses = estimate_loss()
        print(f"step:{iter},train losses:{losses['train']:.3f} val losses: {losses['val']:.3f}")
    #sample a batch of data
    xb, yb = get_batch('train')

    logits,loss = model.forward(xb,yb)
    optimizer.zero_grad(set_to_none=True)        #mot for RNN's, previously accumulated gradients wont affect current training if set_to_none is True
    loss.backward()
    optimizer.step()

print(loss.item())

step:0,train losses:2.655 val losses: 2.682
step:1000,train losses:2.633 val losses: 2.661
step:2000,train losses:2.603 val losses: 2.633
step:3000,train losses:2.592 val losses: 2.621
step:4000,train losses:2.573 val losses: 2.598
step:5000,train losses:2.561 val losses: 2.600
step:6000,train losses:2.539 val losses: 2.585
step:7000,train losses:2.540 val losses: 2.583
step:8000,train losses:2.541 val losses: 2.560
step:9000,train losses:2.512 val losses: 2.556
2.455850839614868


In [69]:
context = torch.zeros((1,1), dtype = torch.long, device=device)
generated_chars = decode(m.generate(context,max_new_tokens=500)[0].tolist())
print(generated_chars)



BzP&Bmjt t thanwad.
_I th
sa mifingge snd ad bon grid sh:Mcoutu d ant yerlas st
"Wnour pu IAPAate;L*
"Aurerdd h
d m wicut l B[j0VE. a y

" jKpe shemor u, atid any n a, are crmillithodbothy.Oza h.
eonss cls e ut avofth ils ou y thhHed frondnthear  brld bus: d Whemun bleb DE4TEpe " wer
forode 6res.
sar ingsoind ors medore  thtin;din h he s t I]3Oin-Hbu therops thesewr suthy,-'res,"I the caylowQ! s.

"B(JWen he unore o;;;ld t'carnds."be fan."
Bluriller t t tontmited bre ary st ect-M*"
the cr.
ender
